# Feature Extraction for EMG signals

## using LSTM --->  TRAINING + EVALUATION + EXTRACTION

In [45]:
import torch
import torch.nn as nn
import torch.optim as optim
from models import LSTM_Emb_Classifier, EMG_Feature_Extractor
from utils.loaders import ActionNetEmgDataset
from torch.utils.data import DataLoader
from utils.logger import logger
from tqdm import tqdm
import pickle
import os
from utils.loaders import FeaturesExtendedEMGDataset
from models import MLP

### Training [ + EVALUATION ]

In [56]:
BATCH_SIZE = 32
LR = 0.001
MOMENTUM = 0.9
WEIGHT_DECAY = 1e-4
STEP_SIZE = 20
GAMMA = 0.1
NUM_EPOCHS = 50

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
if torch.backends.mps.is_available():
    DEVICE = 'mps'
    logger.info("------ USING APPLE SILICON GPU ------")
 

2024-06-11 21:01:05 LOG INFO ------ USING APPLE SILICON GPU ------


In [57]:
# Parametri del modello
input_dim = 16
hidden_dim = 300
embedding_dim = 256
output_dim = 20  # Definisci il numero di classi

# Inizializzazione del modello, della loss function e dell'ottimizzatore
model = LSTM_Emb_Classifier(input_dim=input_dim, hidden_dim=hidden_dim, embedding_dim=embedding_dim, num_class=output_dim)
model = model.to(DEVICE)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=STEP_SIZE, gamma=GAMMA)

train_dataset = ActionNetEmgDataset('train', 25, 5, True, './action-net', "./action-net/saved_emg", 2) # Inserisci il path al dataset di training
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=True, drop_last=True) # Inserisci il dataloader per il training

val_dataset = ActionNetEmgDataset('test', 25, 5, True, './action-net', "./action-net/saved_emg", 2)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=True, drop_last=True)

logger.info(f"Model: {model}")
logger.info(f"len train_dataset: {len(train_dataset)}")
logger.info(f"len train_loader: {len(train_loader)}")


/Users/andreavannozzi/GithubProjects/Multimodal-Egocentric-Action-Recognition/env/lib/python3.8/site-packages/torch/nn/modules/rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x15a01c0d0>
Traceback (most recent call last):
  File "/Users/andreavannozzi/GithubProjects/Multimodal-Egocentric-Action-Recognition/env/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1479, in __del__
    self._shutdown_workers()
  File "/Users/andreavannozzi/GithubProjects/Multimodal-Egocentric-Action-Recognition/env/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1437, in _shutdown_workers
    if self._persistent_workers or self._workers_status[worker_id]:
AttributeError: '_MultiProcessingDataLoaderIter' obj

In [60]:
def evaluate(model, data_loader, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for x, y in data_loader:
            x = x.reshape(BATCH_SIZE, 5, 25, -1)
            x = x.permute(1, 0, 2, 3)
            y = y.to(device)
            
            for i in range(5):
                x_t = x[i].float().to(device)
                outputs, embeddings = model(x_t)
                _, predicted = torch.max(outputs, 1)
                total += y.size(0)
                correct += (predicted == y).sum().item()

    accuracy = correct / total
    return accuracy


In [ ]:
model.load_state_dict(torch.load('./saved_models/LSTM_Emb_Classifier/NEW_final_LSTM_Emb_25_epoch_20.pth'))
model.train()
for epoch in range(NUM_EPOCHS):
        model.train()
        epoch_loss = [0.0, 0]
        for i_val,(x, y) in tqdm(enumerate(train_loader)):
            x = x.reshape(BATCH_SIZE, 5, 25, -1)
            x = x.permute(1, 0, 2, 3)
            y = y.to(DEVICE)
            #logger.info(f"X: {x[0][0]}")
            # Category Loss
            #logger.info(f"X: {x.size()}")
            for i in range(5):
                x_t = x[i].float().to(DEVICE)
                outputs, embeddings = model(x_t)
                # Log details about the outputs
                #logger.info(f"Output type: {cls_o.logits.shape}")
                
                criterion = nn.CrossEntropyLoss()
                loss = criterion(outputs, y.long())

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                epoch_loss[0] += loss.item()
                epoch_loss[1] += x.size(0)

                if (i_val + 1) % (len(train_loader) // 5) == 0:
                    logger.info("[{}/{}]".format(i_val + 1, len(train_loader)))
            
        scheduler.step()
        logger.info(f'[EPOCH {epoch+1}] Avg. Loss: {epoch_loss[0] / epoch_loss[1]}')


        #save checkpoint in a file
        if (epoch+1) % 10 == 0:
            train_accuracy = evaluate(model, train_loader, DEVICE)
            val_accuracy = evaluate(model, val_loader, DEVICE)
            logger.info(f'[EPOCH {epoch+1}] Train Accuracy: {train_accuracy}')
            logger.info(f'[EPOCH {epoch+1}] Val Accuracy: {val_accuracy}')
            torch.save(model.state_dict(), f'./saved_models/LSTM_Emb_Classifier/NEW_final_LSTM_Emb_25_epoch_{epoch+1}.pth')
        if (epoch+1) % STEP_SIZE == 0:
            logger.info(f'Current LR: {scheduler.get_last_lr()}')

### Extraction

In [67]:
train_dataset = ActionNetEmgDataset('train', 25, 5, True, './action-net', "./action-net/saved_emg", 2) # Inserisci il path al dataset di training
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True, num_workers=4, pin_memory=True, drop_last=True) # Inserisci il dataloader per il training

val_dataset = ActionNetEmgDataset('test', 25, 5, True, './action-net', "./action-net/saved_emg", 2)
val_loader = DataLoader(val_dataset, batch_size=1, shuffle=True, num_workers=4, pin_memory=True, drop_last=True)

model.load_state_dict(torch.load(f'./saved_models/LSTM_Emb_Classifier/NEW_final_LSTM_Emb_25_epoch_10.pth'))
model.eval()

embeddings_list = []
labels_list = []

with torch.no_grad():
    for i_val,(x, y) in tqdm(enumerate(train_loader)):
        x = x.reshape(1, 5, 25, -1)
        x = x.permute(1, 0, 2, 3)
        y = y.to(DEVICE)
        for i in range(5):
                x_t = x[i].float().to(DEVICE)
                sample = {}
                outputs, embeddings = model(x_t)
                embeddings_list.append(embeddings.cpu())
                labels_list.append(y.cpu())

# Combina le features e le labels in una lista di dizionari
data_to_save = [{'features': emb[0], 'labels': lbl} for emb, lbl in zip(embeddings_list, labels_list)]
# print size of the data
print(f"Data size: {len(data_to_save)}")
# Salva i dati in un file pickle
'''
features_file = "saved_features/EMG_Emb_LSTM_25_dense_D1_train.pkl"
with open(features_file, 'wb') as f:
    pickle.dump(data_to_save, f)
'''

1795it [00:44, 40.79it/s] 

Data size: 8975


'\nfeatures_file = "saved_features/EMG_Emb_LSTM_25_dense_D1_train.pkl"\nwith open(features_file, \'wb\') as f:\n    pickle.dump(data_to_save, f)\n'

In [68]:
sample = data_to_save[0]
print(f"Data size: {sample['features'].size()}")

features_file = "saved_features/NEW_EMG_Emb_LSTM_25_dense_D1_train.pkl"
with open(features_file, 'wb') as f:
    pickle.dump(data_to_save, f)

Data size: torch.Size([256])


## EGM Feature Extraction using Signal Properties such as Integrated EMG, Mean Squared Value, Variance, Root Mean Square, Kurtosis

In [ ]:
from models import EMG_Feature_Extractor


train_dataset = ActionNetEmgDataset('test', 25, 5, True, './action-net', "./action-net/saved_emg", 2) # Inserisci il path al dataset di training
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True, num_workers=4, pin_memory=True, drop_last=True) # Inserisci il dataloader per il training

embeddings_list = []
labels_list = []

for i_val,(x, y) in tqdm(enumerate(train_loader)):
            x = x.reshape(1, 5, 25, -1)
            x = x.permute(1, 0, 2, 3)
            y = y.to(DEVICE)
            #logger.info(f"X: {x[0][0]}")
            # Category Loss
            #logger.info(f"X: {x.size()}")
            for i in range(5):
                sample = {}
                x_t = x[i].float().to(DEVICE)
                embeddings = EMG_Feature_Extractor(x_t[0])
                embeddings_list.append(embeddings.cpu())  # Mantenere gli embeddings sulla CPU per salvare
                labels_list.append(y.cpu())  # Mantenere le etichette sulla CPU per salvare

data_to_save = [{'features': emb, 'labels': lbl} for emb, lbl in zip(embeddings_list, labels_list)]

features_file = "saved_features/EMG_Emb_Stat_25_dense_D1_test.pkl"
with open(features_file, 'wb') as f:
    pickle.dump(data_to_save, f)




# EPIC Kitchens EMG Extraction

In [69]:
import torch
from torch.optim import Adam
from torch.utils.data import DataLoader
from utils.loaders import FeaturesExtendedDataset, FeaturesExtendedEMGDataset, ActionNetEmgRgbDataset
from models import I3D
from models import EMG_Feature_Extractor
from utils.args import args
from omegaconf import OmegaConf
import tqdm
import pickle
from utils.loaders import FeaturesDataset

from models import FC_VAE, LSTM_Emb_Classifier, EMG_Feature_Extractor
from train_vae import train_tuning, evaluate_tuning

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
if torch.backends.mps.is_available():
    DEVICE = torch.device("mps")
    print("------ USING APPLE SILICON GPU ------")

------ USING APPLE SILICON GPU ------


In [70]:
BATCH_SIZE = 1

train_dataset = FeaturesDataset("./saved_features/saved_feat_I3D_25_dense_D1",'train')
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, drop_last=False)

test_dataset = FeaturesDataset("./saved_features/saved_feat_I3D_25_dense_D1",'test')
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, drop_last=False)
print(f"Train dataset size: {len(train_dataset)}")
print(f"Test dataset size: {len(test_dataset)}")

Train dataset size: 1543
Test dataset size: 435


In [72]:
# Inizializzazione del modello, della loss function e dell'ottimizzatore
model_finetune = FC_VAE(dim_input=1024, nz=64, dim_output=256)
model_finetune.to(DEVICE)

checkpoint = torch.load('./saved_models/VAE_Fine_Tuninng/VAE_RGB_to_EMG_epoch_50.pth', map_location=DEVICE)
model_finetune.load_state_dict(checkpoint)

model_finetune.eval()

emb_train = []
lab_train = []
emb_test = []
lab_test = []

with torch.no_grad():

    for (idx_train ,(x_train, y_train)) in enumerate(train_loader):
        x_train = x_train.reshape(5, 1, 1024)
        x_train_t = x_train.float().to(DEVICE)
        train_clips = []
        for i in range(5):
            embeddings_train, latents, mu, logvar = model_finetune(x_train_t[i])
            train_clips.append(embeddings_train[0].cpu().detach().numpy())

        lab_train.append(y_train[0].cpu().detach().numpy())
        emb_train.append(train_clips)

    for (idx_test ,(x_test, y_test)) in enumerate(test_loader):
        x_test = x_test.reshape(5, 1, 1024)
        x_test_t = x_test.float().to(DEVICE)
        test_clips = []
        for i in range(5):
            embeddings_test, latents, mu, logvar = model_finetune(x_test_t[i])
            test_clips.append(embeddings_test[0].cpu().detach().numpy())
        lab_test.append(y_test[0].cpu().detach().numpy())
        emb_test.append(test_clips)

                            

print(f"Embeddings train size: {len(emb_train)}")
print(f"Labels train size: {len(lab_train)}")
print(f"Embeddings test size: {len(emb_test)}")
print(f"Labels test size: {len(lab_test)}")


        


Embeddings train size: 1543
Labels train size: 1543
Embeddings test size: 435
Labels test size: 435


In [31]:
lab_train

[array(5),
 array(3),
 array(0),
 array(2),
 array(1),
 array(6),
 array(4),
 array(0),
 array(2),
 array(1),
 array(6),
 array(4),
 array(2),
 array(2),
 array(0),
 array(4),
 array(0),
 array(0),
 array(4),
 array(2),
 array(1),
 array(4),
 array(0),
 array(1),
 array(1),
 array(0),
 array(2),
 array(0),
 array(4),
 array(0),
 array(3),
 array(0),
 array(0),
 array(0),
 array(1),
 array(0),
 array(1),
 array(4),
 array(0),
 array(3),
 array(0),
 array(4),
 array(1),
 array(1),
 array(0),
 array(0),
 array(2),
 array(2),
 array(1),
 array(4),
 array(0),
 array(1),
 array(4),
 array(5),
 array(2),
 array(0),
 array(4),
 array(0),
 array(0),
 array(4),
 array(4),
 array(1),
 array(2),
 array(2),
 array(1),
 array(5),
 array(5),
 array(1),
 array(0),
 array(0),
 array(2),
 array(0),
 array(0),
 array(2),
 array(0),
 array(0),
 array(2),
 array(4),
 array(3),
 array(2),
 array(2),
 array(4),
 array(0),
 array(0),
 array(1),
 array(1),
 array(0),
 array(6),
 array(0),
 array(1),
 array(1),

In [74]:
train_file = "saved_features/EPIC_KITCHENS_EMG_LSTM_25_dense_D1_train.pkl"
test_file = "saved_features/EPIC_KITCHENS_EMG_LSTM_25_dense_D1_test.pkl"

data_to_save_train = [{'features': emb, 'labels': lbl} for emb, lbl in zip(emb_train, lab_train)]
data_to_save_test = [{'features': emb, 'labels': lbl} for emb, lbl in zip(emb_test, lab_test)]

with open(train_file, 'wb') as f:
    pickle.dump(data_to_save_train, f)

with open(test_file, 'wb') as f:
    pickle.dump(data_to_save_test, f)